In [2]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd

directory = r"C:\Users\doris\Downloads\dataset"

# Collect all JSON file paths
records = []
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.lower().endswith(".json"):
            records.append({
                "json_name": file,
                "directory": root
            })

# Convert to DataFrame
df = pd.DataFrame(records)

# Save to Excel on Desktop
output_file = r"C:\Users\doris\Desktop\IKT Project\json_files.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved {len(df)} JSON file records to {output_file}")


Saved 22068 JSON file records to C:\Users\doris\Desktop\IKT Project\json_files.xlsx


In [8]:
import pandas as pd
import sqlite3

# Load the Excel files
dataset_info = pd.read_excel(r"C:\Users\doris\Desktop\IKT Project\dataset_info_post.xlsx", sheet_name="Sheet1")
json_files = pd.read_excel(r"C:\Users\doris\Desktop\IKT Project\json_files.xlsx")

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Write both DataFrames into the database
dataset_info.to_sql("dataset_info", conn, index=False, if_exists="replace")
json_files.to_sql("json_files", conn, index=False, if_exists="replace")

# Example SQL JOIN (adjust join column names as needed)
query = """
SELECT 
    jf.*,di.*
FROM json_files jf
LEFT JOIN dataset_info di
on jf.json_name = di.filename

"""

result = pd.read_sql(query, conn)

print(result.head())


                                       json_name  \
0  guatemala-volcano_00000004_post_disaster.json   
1   guatemala-volcano_00000004_pre_disaster.json   
2  guatemala-volcano_00000012_post_disaster.json   
3   guatemala-volcano_00000012_pre_disaster.json   
4  guatemala-volcano_00000014_post_disaster.json   

                                           directory  \
0  C:\Users\doris\Downloads\dataset\geotiffs\hold...   
1  C:\Users\doris\Downloads\dataset\geotiffs\hold...   
2  C:\Users\doris\Downloads\dataset\geotiffs\hold...   
3  C:\Users\doris\Downloads\dataset\geotiffs\hold...   
4  C:\Users\doris\Downloads\dataset\geotiffs\hold...   

                                        filename disaster_type  items  \
0  guatemala-volcano_00000004_post_disaster.json       volcano   20.0   
1                                           None          None    NaN   
2  guatemala-volcano_00000012_post_disaster.json       volcano   33.0   
3                                           None          

In [9]:
import pandas as pd
import sqlite3

# Load the Excel files
dataset_info = pd.read_excel(r"C:\Users\doris\Desktop\IKT Project\dataset_info_post.xlsx", sheet_name="Sheet1")
json_files = pd.read_excel(r"C:\Users\doris\Desktop\IKT Project\json_files.xlsx")

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Write both DataFrames into the database
dataset_info.to_sql("dataset_info", conn, index=False, if_exists="replace")
json_files.to_sql("json_files", conn, index=False, if_exists="replace")

# Example SQL JOIN (adjust join keys if needed)
query = """
SELECT 
    jf.*,di.*
FROM json_files jf
LEFT JOIN dataset_info di
on jf.json_name = di.filename

"""

# Run query
result = pd.read_sql(query, conn)

# Save result to a new Excel file
output_file = r"C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx"
result.to_excel(output_file, index=False)

print(f"Joined result saved to {output_file}")


Joined result saved to C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx


In [11]:
import pandas as pd

# Path to the existing file
file_path = r"C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx"

# Load the Excel file
df = pd.read_excel(file_path)

# Create new column safely
df["divided_value"] = df.apply(
    lambda row: row["weighted_m2"] / row["area_m2"] 
    if pd.notnull(row["weighted_m2"]) and pd.notnull(row["area_m2"]) and row["area_m2"] != 0 
    else None, 
    axis=1
)

# Overwrite the same Excel file
df.to_excel(file_path, index=False)

print(f"Column 'divided_value' added and file updated: {file_path}")



Column 'divided_value' added and file updated: C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx


In [14]:

# Add new column based on json_name
df["pre_post_flag"] = df["json_name"].apply(
    lambda x: "post" if isinstance(x, str) and "post" in x.lower()
    else ("pre" if isinstance(x, str) and "pre" in x.lower()
    else None)
)

# Overwrite the same Excel file
df.to_excel(file_path, index=False)

print(f"Column 'pre_post_flag' added and file updated: {file_path}")

Column 'pre_post_flag' added and file updated: C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx


In [15]:
# Extract base id (remove _pre_disaster.json / _post_disaster.json)
df["base_id"] = df["json_name"].str.replace(r"_(pre|post)_disaster\.json$", "", regex=True)

# Overwrite the same Excel file
df.to_excel(file_path, index=False)

print(f"Column 'base_id' added and file updated: {file_path}")

Column 'base_id' added and file updated: C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx


In [25]:
import pandas as pd
import sqlite3

# Load the Excel file
prepost = pd.read_excel(r"C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx", sheet_name="Sheet1")

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Save the DataFrame to SQLite with a simple table name
prepost.to_sql("prepost", conn, index=False, if_exists="replace")

# Optional: check available tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:\n", tables)

# Query the table
query = """
WITH ranked AS (
    SELECT
        disaster_type,
        CAST(divided_value AS INT) AS floor_int,
        t.*,
        ROW_NUMBER() OVER (
            PARTITION BY disaster_type, CAST(divided_value AS INT)
            ORDER BY divided_value DESC
        ) AS rn
    FROM prepost t
    where divided_value <> 0 and divided_value is not null
)
SELECT *
FROM ranked
WHERE rn <= 1000
ORDER BY disaster_type, floor_int, rn;

"""
result = pd.read_sql(query, conn)

# Show first rows
print(result.head())

# Close the connection when done
conn.close()


Tables in DB:
       name
0  prepost
  disaster_type  floor_int                                      json_name  \
0    earthquake          0  mexico-earthquake_00000181_post_disaster.json   
1    earthquake          0  mexico-earthquake_00000076_post_disaster.json   
2    earthquake          0  mexico-earthquake_00000053_post_disaster.json   
3    earthquake          0  mexico-earthquake_00000095_post_disaster.json   
4    earthquake          0  mexico-earthquake_00000059_post_disaster.json   

                                           directory  \
0  C:\Users\doris\Downloads\dataset\geotiffs\tier...   
1  C:\Users\doris\Downloads\dataset\geotiffs\test...   
2  C:\Users\doris\Downloads\dataset\geotiffs\tier...   
3  C:\Users\doris\Downloads\dataset\geotiffs\test...   
4  C:\Users\doris\Downloads\dataset\geotiffs\tier...   

                                        filename disaster_type:1  items  \
0  mexico-earthquake_00000181_post_disaster.json      earthquake  368.0   
1  mexico-ear

In [27]:
import pandas as pd
import sqlite3

# File path
excel_file = r"C:\Users\doris\Desktop\IKT Project\Pre&Post_dataset.xlsx"

# Load the Excel file
prepost = pd.read_excel(excel_file, sheet_name="Sheet1")

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Save the DataFrame to SQLite with a simple table name
prepost.to_sql("prepost", conn, index=False, if_exists="replace")

# Optional: check available tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:\n", tables)

# Query the table
query = """
WITH ranked AS (
    SELECT
        disaster_type,
        CAST(divided_value AS INT) AS floor_int,
        t.*,
        ROW_NUMBER() OVER (
            PARTITION BY disaster_type, CAST(divided_value AS INT)
            ORDER BY divided_value DESC
        ) AS rn
    FROM prepost t
    WHERE divided_value <> 0 
      AND divided_value IS NOT NULL
)
SELECT *
FROM ranked
WHERE rn <= 1000
ORDER BY disaster_type, floor_int, rn;
"""
result = pd.read_sql(query, conn)

# Show first rows
print(result.head())

# Close the SQLite connection
conn.close()

# Write result back into the same Excel file, replacing "selected" sheet if exists
with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    result.to_excel(writer, sheet_name="selected", index=False)

print("✅ Results written to 'selected' sheet in Excel.")


Tables in DB:
       name
0  prepost
  disaster_type  floor_int                                      json_name  \
0    earthquake          0  mexico-earthquake_00000181_post_disaster.json   
1    earthquake          0  mexico-earthquake_00000076_post_disaster.json   
2    earthquake          0  mexico-earthquake_00000053_post_disaster.json   
3    earthquake          0  mexico-earthquake_00000095_post_disaster.json   
4    earthquake          0  mexico-earthquake_00000059_post_disaster.json   

                                           directory  \
0  C:\Users\doris\Downloads\dataset\geotiffs\tier...   
1  C:\Users\doris\Downloads\dataset\geotiffs\test...   
2  C:\Users\doris\Downloads\dataset\geotiffs\tier...   
3  C:\Users\doris\Downloads\dataset\geotiffs\test...   
4  C:\Users\doris\Downloads\dataset\geotiffs\tier...   

                                        filename disaster_type:1  items  \
0  mexico-earthquake_00000181_post_disaster.json      earthquake  368.0   
1  mexico-ear